In [4]:
### IMPORTANT:

### BECAUSE THIS PROJECT REQUIRES AN ENORMOUS AMOUNT OF TRAINING TIME 
### (YEARS ON A SINGLE GPU)
### THIS CODE WILL NOT STOP RUNNING UNTIL YOU STOP IT, OR IT RUNS OUT OF RAM!

### ENABLE GPU ON SYSTEM, IF AVAILABLE, 
### OR RUNNING JUST ONE TRAINING ITERATION WILL TAKE FOREVER.
### THIS PROGRAM USES A *LOT* OF DATA

### - Aaron

###

In [ ]:
### library to pickle and compress arrays simultaneously
### In it's own cell, so you can be aware that this code 
### requires this library that you likely don't have yet.
!pip install compress_pickle

In [5]:
import numpy as np
import scipy as sp
from scipy.ndimage import correlate
import pandas as pd
import time
import math
import random
import multiprocessing
from datetime import datetime
import sys
import networkx as nx
from collections.abc import Iterable
# import pickle
from random import sample
import matplotlib.pyplot as plt
import seaborn as sns
import compress_pickle as pickle
from keras.layers import Dense, Add, Concatenate, Conv2D, Input, MaxPooling2D, Conv2DTranspose, Flatten, Masking, Reshape, LayerNormalization, Softmax, Activation, GaussianNoise
from keras import Model
# import tensorflow as tf
import os
import keras
import keras.backend as K

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
class MultiPente():
  def __init__(self, n_players=2, n_games=100, agent=None, graph_path=None):
    self.boards = np.zeros(shape=(n_games, 19, 19)).astype(int)
    self.prev_boards = np.zeros(shape=(n_games, 19, 19)).astype(int)
    self.pairs = np.zeros((n_games, n_players))

    self.agent = agent

    self.n_players = n_players
    self.total_n_games = n_games
    self.n_games = n_games
    self.completed = np.array([False] * n_games)

    self.outcomes = [[],[]]
    
    if graph_path is None:
        self.graph = nx.DiGraph()
    else:
        try:
            with open(graph_path, "rb") as f:
                self.graph = pickle.load(f)
                
            print("----------------------")
            print("LOADED EXISTING GRAPH")
            print("----------------------")
            
        except:
            self.graph = nx.DiGraph()
  
    self.curr_player = 1

    # Create filters for finding surrounded pairs on the board
    self.vertical_pairs = np.array([[[1],
                                    [-1],
                                    [-1],
                                    [1]]])

    self.horizontal_pairs = np.array([[[1, -1, -1, 1]]])

    self.main_diag_pairs = np.array([[[1, 0, 0, 0],
                                      [0, -1, 0, 0],
                                      [0, 0, -1, 0],
                                      [0, 0, 0, 1]]])

    self.off_diag_pairs = np.array([[[0, 0, 0, 1],
                                    [0, 0, -1, 0],
                                    [0, -1, 0, 0],
                                    [1, 0, 0, 0]]])
    
    self.vertical_conv = np.array([[[1],
                                    [1],
                                    [1],
                                    [1],
                                    [1]]])

    self.horizontal_conv = np.array([[[1, 1, 1, 1, 1]]])

    self.main_diag_conv = np.array([[[1, 0, 0, 0, 0],
                                      [0, 1, 0, 0, 0],
                                      [0, 0, 1, 0, 0],
                                      [0, 0, 0, 1, 0],
                                      [0, 0, 0, 0, 1]]])
    
    self.off_diag_conv = np.array([[[0, 0, 0, 0, 1],
                                    [0, 0, 0, 1, 0],
                                    [0, 0, 1, 0, 0],
                                    [0, 1, 0, 0, 0],
                                    [1, 0, 0, 0, 0]]])
    


  def getLegalMoves(self):
    mask = (self.boards == 0)
    return mask

  # def checkRow(self, seq):
  #   winning_seq = np.array([1, 1, 1, 1, 1])
  #   if 5 in np.convolve(winning_seq, seq, "valid"):
  #     return True
  #   else:
  #     return False

  def make_hashable(self, a):
    out = pickle.dumps(a, compression="bz2")
    return out

  def getMoveIndices(self, agent=None, train=True, make_random_moves=False):

    if make_random_moves:
      
      self.values = agent.getValues(self.boards, self.pairs, self.last_player, self.graph, train=train, random_agent=True)
      
      self.mask = self.getLegalMoves()

      self.values += np.random.normal(0, .001, size=self.values.shape)

      self.values *= self.mask


      rng = np.random.default_rng()
      seq = np.vstack([rng.permutation(361).reshape(1,19,19) for i in range(self.values.shape[0])])
      
      self.values *= seq

      self.decision_indices = np.swapaxes(np.array((np.argmax(np.max(self.values, axis=2), axis=1), np.argmax(np.max(self.values, axis=1), axis=1))),0,-1)

      decisions =(np.amax(self.values, axis=(1,2), keepdims=True) == self.values)

      return decisions
    
    else:
      # See http://tim.hibal.org/blog/alpha-zero-how-and-why-it-works/ for more information on UCB scores.

      self.values = agent.getValues(self.boards, self.pairs, self.last_player, self.graph, train=train, random_agent=False)
      
      self.mask = self.getLegalMoves()

      # Adding small amount of noise to work around odd issue 
      # where agent tries to take same move over and over.

      self.values += np.random.normal(0, .001, size=self.values.shape)

      self.values *= self.mask

      rng = np.random.default_rng()
      seq = np.vstack([rng.permutation(361).reshape(1,19,19) for i in range(self.values.shape[0])])

      self.values *= seq

      self.decision_indices = np.swapaxes(np.array((np.argmax(np.max(self.values, axis=2), axis=1), np.argmax(np.max(self.values, axis=1), axis=1))),0,-1)

      decisions = (np.amax(self.values, axis=(1,2), keepdims=True) == self.values)

      return decisions

  def makeMoves(self, player_num, agent, move_num, train=True, make_random_moves=False):

    self.last_player = 0 if player_num == 1 else 1

    # If in training mode, use UCB scores to choose moves, otherwise choose agent predictions
    self.choices = self.getMoveIndices(agent, train=train, make_random_moves=make_random_moves)

    # print(np.sum(self.choices, axis=(1,2)))
    assert np.all(np.isclose(np.sum(self.choices, axis=(1,2)), 1))

    # Cache current game state
    self.prev_boards = self.boards

    # print("MAKING MOVES AND UPDATING BOARD")

    if train:
      # Add predecessor nodes to the graph if they aren't already in the graph
      self.last_nodes = [self.make_hashable((data[0], data[1], data[2])) for data in list(zip(self.prev_boards.astype(int), self.pairs[:, self.last_player].astype(int), self.pairs[:, player_num].astype(int)))]

      # print("LAST NODE:\n")
      # print(list(zip(intermediate_boards.tolist(), self.pairs[:, self.last_player].tolist(), self.pairs[:, player_num].tolist())))

      add_last_nodes = [node for node in self.last_nodes if node not in self.graph]
      # print("NUM ORIGINATING NODES NOT IN GRAPH:", len(add_last_nodes))
      self.graph.add_nodes_from(add_last_nodes, visits=1, outcomes=0.5, player=self.last_player)

    # Make moves based on choices
    self.boards += self.choices

    # All possible cases where a player wins by getting five pieces in a row
    case1 = np.sum(correlate(self.boards, self.vertical_conv, mode="constant") >= 4.5, axis=(1,2))
    case2 = np.sum(correlate(self.boards, self.horizontal_conv, mode="constant") >= 4.5, axis=(1,2))
    case3 = np.sum(correlate(self.boards, self.main_diag_conv, mode="constant") >= 4.5, axis=(1,2))
    case4 = np.sum(correlate(self.boards, self.off_diag_conv, mode="constant") >= 4.5, axis=(1,2))

    # Nested or statements that calculate if any of these cases are True
    finished = np.bitwise_or(np.bitwise_or(np.bitwise_or(case1, case2), case3), case4)
    # print("NUMBER OF GAMES IN PROGRESS:", finished.shape)

    
    # print(vertical_pairs.shape)
    # print(horizontal_pairs.shape)
    # print(main_diag_pairs.shape)
    # print(off_diag_pairs.shape)

    # Find pairs that existed on board prior to moving
    prev_vert_pairs = (correlate(self.prev_boards, self.vertical_pairs, mode="constant") >= 3.5)
    prev_hor_pairs = (correlate(self.prev_boards, self.horizontal_pairs, mode="constant") >= 3.5)
    prev_main_pairs = (correlate(self.prev_boards, self.main_diag_pairs, mode="constant") >= 3.5)
    prev_off_pairs = (correlate(self.prev_boards, self.off_diag_pairs, mode="constant") >= 3.5)

    # Find pairs that exist on board after making moves
    curr_vert_pairs = (correlate(self.boards, self.vertical_pairs, mode="constant") >= 3.5)
    curr_hor_pairs = (correlate(self.boards, self.horizontal_pairs, mode="constant") >= 3.5)
    curr_main_pairs = (correlate(self.boards, self.main_diag_pairs, mode="constant") >= 3.5)
    curr_off_pairs = (correlate(self.boards, self.off_diag_pairs, mode="constant") >= 3.5)

    # Find ONLY the pairs that resulted directly from making the most recent moves
    delta_vert_pairs = np.logical_xor(curr_vert_pairs, prev_vert_pairs)
    delta_hor_pairs = np.logical_xor(curr_hor_pairs, prev_hor_pairs)
    delta_main_pairs = np.logical_xor(curr_main_pairs, prev_main_pairs)
    delta_off_pairs = np.logical_xor(curr_off_pairs, prev_off_pairs)

    remove_vert = np.logical_or(delta_vert_pairs, np.roll(delta_vert_pairs, -1, axis=1))
    remove_hor = np.logical_or(delta_hor_pairs, np.roll(delta_hor_pairs, -1, axis=2))
    remove_main = np.logical_or(delta_main_pairs, np.roll(np.roll(delta_main_pairs, -1, axis=2), -1, axis=1))
    remove_off = np.logical_or(np.roll(delta_off_pairs, -1, axis=1), np.roll(delta_off_pairs, -1, axis=2))

    # assert np.all(np.sum(remove_vert, axis=(1,2)) % 2 == 0)
    # assert np.all(np.sum(remove_hor, axis=(1,2)) % 2 == 0)
    # assert np.all(np.sum(remove_main, axis=(1,2)) % 2 == 0)
    # assert np.all(np.sum(remove_off, axis=(1,2)) % 2 == 0)

    # assert np.all(remove_vert.astype(int) == (-intermediate_boards.astype(int) * remove_vert.astype(int)))
    # assert np.all(remove_hor.astype(int) == (-intermediate_boards.astype(int) * remove_hor.astype(int)))
    # assert np.all(remove_main.astype(int) == (-intermediate_boards.astype(int) * remove_main.astype(int)))
    # assert np.all(remove_off.astype(int) == (-intermediate_boards.astype(int) * remove_off.astype(int)))

    remove_all = remove_vert.astype(int) + remove_hor.astype(int) + remove_main.astype(int) + remove_off.astype(int)

    # assert not np.any(remove_all < 0) and not np.any(remove_all > 1)
    # assert np.all(remove_all == (-intermediate_boards * remove_all).astype(bool))

    self.boards -= remove_all
    # assert np.all(self.boards >= -.05)

    # Update pair count
    self.pairs[:, player_num] += np.sum(remove_all, axis=(1,2)) / 2
    
    # Check if current player won by capturing five or more pairs
    self.completed = np.logical_or(finished, (self.pairs[:, player_num] >= 5))

    # Flip board to be viewed from perspective of other player
    self.boards *= -1

    # print("ADDING RESULTING NODES TO GRAPH")

    if train:
      # Add current nodes to the graph if they aren't already in the graph
      curr_nodes = [self.make_hashable((data[0], data[1], data[2])) for data in list(zip(self.boards.astype(int), self.pairs[:, player_num].astype(int), self.pairs[:, self.last_player].astype(int)))]
      add_curr_nodes = [node for node in curr_nodes if node not in self.graph]
      self.graph.add_nodes_from(add_curr_nodes, visits=1, outcomes=0.5, player=player_num)

      # Add edges between previous game states and corresponding new game state
      edges_with_data = list(zip(self.last_nodes, curr_nodes, self.decision_indices))

      self.graph.add_edges_from([(edge[0], edge[1], {"move":edge[2]}) for edge in edges_with_data])

      # Store node visit count and outcome (won by player 1 or 0) in graph
      completed_nodes_data = list(zip(self.boards[self.completed].astype(int), self.pairs[:, player_num][self.completed].astype(int), self.pairs[:, self.last_player][self.completed].astype(int)))
      completed_nodes = [self.make_hashable((data[0], data[1], data[2])) for data in completed_nodes_data]

      for node in completed_nodes:
        ancestors = nx.ancestors(self.graph, node)
        # print("NUM ANCESTORS BEING UPDATED:", len(ancestors))
        for node in ancestors:
          self.graph.nodes[node]["visits"] += 1
          self.graph.nodes[node]["outcomes"] += int(player_num == self.graph.nodes[node]["player"])

    # Update outcome records
    self.outcomes[player_num] += [move_num]*sum(self.completed)

    # Drop all completed games from current computation
    self.boards = self.boards[np.logical_not(self.completed)]
    self.pairs = self.pairs[np.logical_not(self.completed)]
    self.n_games = self.pairs.shape[0]

    if self.n_games != self.total_n_games:
      self.completed = np.array([False] * self.n_games)
    # print("SHAPE OF CURRENT BOARDS MATRIX AFTER DROPPING COMPLETED GAMES:", self.boards.shape)
### 
          
    return self.completed  

  def getBoards(self):
    return self.boards

  def play_train(self, agent0, agent1, make_random_moves=[False, False], num_games=100):

    self.total_n_games = num_games
    self.reset()

    count = 0
    player = -1
    while not np.sum(self.completed) == self.n_games and count < 299:

      if player < self.n_players - 1:
        player+=1
      else:
        player=0

      if make_random_moves[0]:
        if player==0:
          fin_list = self.makeMoves(player, agent0, count, make_random_moves=True) 
      else:
        if player==0:
          fin_list = self.makeMoves(player, agent0, count)

      if make_random_moves[1]:
        if player==1:
          fin_list = self.makeMoves(player, agent1, count, make_random_moves=True)
      else:
        if player==1:
          fin_list = self.makeMoves(player, agent1, count)

      print("TAKING TURN", count)
      count += 1

    print("DONE PLAYING", self.total_n_games, "PENTE GAMES")

    # agent0.fit_on_graph(self.graph)

    print("==================")
    print("Player 0 won ", len(self.outcomes[0])/self.total_n_games * 100, "percent of games")
    print("Player 1 won ", len(self.outcomes[1])/self.total_n_games * 100, "percent of games")
    print("==================")    

    self.reset()

  def play_test(self, agent0, agent1, make_random_moves=[False, False], num_games=100):

    self.total_n_games = num_games
    self.reset()

    count = 0
    player = -1
    while not np.sum(self.completed) == self.n_games and count < 299:

      if player < self.n_players - 1:
        player+=1
      else:
        player=0

      # if make_random_moves[0]:
      #   if player==0:
      #     fin_list = self.makeMoves(player, agent0, count, make_random_moves=True) 
      # else:
      #   if player==0:
      #     fin_list = self.makeMoves(player, agent0, count)

      # if make_random_moves[1]:
      #   if player==1:
      #     fin_list = self.makeMoves(player, agent1, count, make_random_moves=True)
      # else:
      #   if player==1:
      #     fin_list = self.makeMoves(player, agent1, count)

      print("TAKING TURN", count)
      count += 1

    print("DONE PLAYING", self.total_n_games, "PENTE GAMES")

    # agent0.fit_on_graph(self.graph)

    print("=========TEST GAMES=========")
    print("Player 0 won ", len(self.outcomes[0])/self.total_n_games * 100, "percent of games")
    print("Player 1 won ", len(self.outcomes[1])/self.total_n_games * 100, "percent of games")
    print("=========TEST GAMES=========")    

    outcome0, outcome1 = len(self.outcomes[0])/self.total_n_games * 100, len(self.outcomes[1])/self.total_n_games * 100
    self.reset()

    return outcome0, outcome1

  def reset(self):
    self.n_games = self.total_n_games
    self.boards = np.zeros(shape=(self.n_games, 19, 19))
    self.pairs = np.zeros((self.n_games, self.n_players))
    self.completed = [False for i in range(self.n_games)]
    self.curr_player = 1
    self.outcomes = [[],[]]

In [7]:
class Agent():
  def __init__(self, agent_path = None):

    self.node_values = {}
    self.scale_factor = 1

    if agent_path is None:
      inp1 = Input(shape=(19,19,1))
      inp2 = Input(shape=(2,))
      # inp_mask = Input(shape=(19,19,1))

      noise = GaussianNoise(.1)(inp1)

      # Convolve
      conv1 = Conv2D(128, 3, activation="relu")(noise)
      noise1 = GaussianNoise(.1)(conv1)
      conv2 = Conv2D(64, 3, activation="relu")(noise1)
      noise2 = GaussianNoise(.1)(conv2)
      conv3 = Conv2D(32, 3, activation="relu")(noise2)
      noise3 = GaussianNoise(.1)(conv3)
      conv4 = Conv2D(16, 3, activation="relu")(noise3)
      noise4 = GaussianNoise(.1)(conv4)
      conv5 = Conv2D(16, 3, activation="relu")(noise4)

      flat1 = Flatten()(conv5)
      encoded = Dense(398, activation="relu")(flat1)
      emb = keras.layers.Embedding(2, 4)(inp2)
      flat_emb = Flatten()(emb)
      cat = Concatenate()([encoded, flat_emb])
      dense3 = Dense(256, activation="relu")(cat)
      dense4 = Dense(400, activation="relu")(dense3)
      norm = LayerNormalization()(dense4)
      reshape = Reshape(target_shape=(5,5,16))(norm)

      # Deconvolve

      deconv5 = Conv2DTranspose(16, 3, activation="relu")(reshape)
      noise5 = GaussianNoise(.1)(deconv5)
      deconv4 = Conv2DTranspose(16, 3, activation="relu")(noise5)
      noise6 = GaussianNoise(.1)(deconv4)
      deconv3 = Conv2DTranspose(16, 3, activation="relu")(noise6)
      noise7 = GaussianNoise(.1)(deconv3)
      deconv2 = Conv2DTranspose(32, 3, activation="relu")(noise7)
      skip2 = Add()([deconv2, conv3])
      deconv1 = Conv2DTranspose(64, 3, activation="relu")(skip2)
      noise8 = GaussianNoise(.1)(deconv1)
      deconv0 = Conv2DTranspose(8, 3, activation="relu")(noise8)
      noise9 = GaussianNoise(.1)(deconv0)
      out = Conv2DTranspose(1, 3, activation="sigmoid")(noise9)

      policy_head = Masking(mask_value=.5)(out)

      # add = Add()([final_deconv * inp_mask, -1*K.cast(inp_mask==0, "float32")])

      # # mask = Masking(mask_value=-1)(add)

      # mask = K.not_equal(add, -1)

      # norm13 = LayerNormalization()(final_deconv)
      # policy_head = keras.layers.Softmax(name="policy_head")(norm13, mask=mask)

      # # Network head for the value of the current (parent) game state
      # dense5 = Dense(256, activation="relu")(dense3)
      # norm11 = LayerNormalization()(dense5)
      # dense6 = Dense(128, activation="relu")(norm11)
      # value_head = Dense(1, activation="sigmoid", name="value_head")(dense6)

      self.model = Model([inp1, inp2], policy_head)

      # Scale up loss to make changes more easily visible during training
      self.model.compile(loss="mse", optimizer="adam")
      # self.model.compile(loss="categorical_crossentropy", optimizer="adam")
      self.model.summary()
    else:
      self.model = keras.models.load_model(agent_path)

    # Hyperparameter for MCTS tuning
    self.c = 10

  def fit_on_graph(self, graph, num_epochs = 25, batch_size = 256, sample_size = 1000):

    for epoch in range(num_epochs):

      boards = []
      pairs = []
      masks = []
      outcome_train = []
      values = []


      for data in sample(graph.nodes(data=True), sample_size):
        node = data[0]

        if node in self.node_values.keys():

          outcomes = data[1]["outcomes"]
          visits = data[1]["visits"]
          player_num = data[1]["player"]
          value = outcomes / visits if visits != 0 else .1

          (board, cur_pairs, prev_pairs) = pickle.loads(node, compression="bz2")

          l = [(edge[0], tuple(edge[2]["move"]), self.get_node_value(graph, edge[1]), graph.nodes(data=True)[edge[1]]["visits"]) for edge in list(graph.out_edges(node, data=True))]

          for data in l:

            if data[0] not in self.node_values.keys():

              self.node_values[data[0]] = (np.zeros(shape=(19,19)), np.zeros(shape=(19,19)), np.zeros(shape=(19,19)))
              
              # print("DATA[1]", data[1])


            # Add outcome score for child
            self.node_values[data[0]][0][data[1]] = data[2]
            # Add 1 to indicate that a node has been visited for masking training later
            self.node_values[data[0]][1][data[1]] = 1
            # Update visit count for particular child node
            self.node_values[data[0]][2][data[1]] = data[3]

          if np.any(self.node_values[node][2] != 0):

            masks.append(self.node_values[node][1])
            outcome_train.append(self.scale_factor * self.node_values[node][0])
            boards.append(board)
            pairs.append([cur_pairs, prev_pairs])
            values.append([value])

      masks = np.array(masks)
      pairs = np.array(pairs)
      values = np.array(values)
      outcome_train = np.array(outcome_train)
      outcome_train = np.reshape(outcome_train, (outcome_train.shape[0], 19, 19, 1))

      # Replace all nan values with 0 that resulted by dividing by zero earlier
      # print(np.sum(outcome_train[~np.isnan(outcome_train)]))
      # t = outcome_train[~np.isnan(outcome_train)].shape[0]*19*19
      # print(np.sum(outcome_train[~np.isnan(outcome_train)]) / t)
      # sys.exit()
      outcome_train[np.isnan(outcome_train)] = 0

      boards = np.array(boards)



      # Train only on data where "masks" has some non-zero elements (i.e. has some explored positions)
      temp = (np.sum(masks, axis=(1,2)) != 0)

      if np.sum(temp) != 0:
          
        temp_masks = masks[temp, :, :]
        temp_boards = boards[temp, :, :]
        temp_outcome = outcome_train[temp, :, :, :]
        temp_values = values[temp, :]
        temp_pairs = pairs[temp, :]

        # Data augmentation with horizontal/vertical flip of boards and transposing

        temp_masks = np.vstack((temp_masks, np.flip(temp_masks, axis=1), 
                                np.transpose(np.flip(temp_masks, axis=1), (0,2,1)), 
                                np.transpose(temp_masks, (0,2,1)), np.flip(temp_masks, axis=2), 
                                np.transpose(np.flip(temp_masks, axis=2), (0,2,1)), 
                                np.transpose(np.flip(np.flip(temp_masks, axis=2), axis=1), (0,2,1)), 
                                np.flip(np.flip(temp_masks, axis=2), axis=1)))
        
        temp_boards = np.vstack((temp_boards, np.flip(temp_boards, axis=1), 
                                np.transpose(np.flip(temp_boards, axis=1), (0,2,1)), 
                                np.transpose(temp_boards, (0,2,1)), np.flip(temp_boards, axis=2), 
                                np.transpose(np.flip(temp_boards, axis=2), (0,2,1)), 
                                np.transpose(np.flip(np.flip(temp_boards, axis=2), axis=1), (0,2,1)), 
                                np.flip(np.flip(temp_boards, axis=2), axis=1)))
        print(temp_outcome.shape)
        temp_outcome = np.vstack((temp_outcome, np.flip(temp_outcome, axis=1), 
                                np.transpose(np.flip(temp_outcome, axis=1), (0,2,1,3)), 
                                np.transpose(temp_outcome, (0,2,1,3)), np.flip(temp_outcome, axis=2), 
                                np.transpose(np.flip(temp_outcome, axis=2), (0,2,1,3)), 
                                np.transpose(np.flip(np.flip(temp_outcome, axis=2), axis=1), (0,2,1,3)), 
                                np.flip(np.flip(temp_outcome, axis=2), axis=1)))
        
        temp_values = np.repeat(temp_values, 8, axis=0)
        temp_pairs = np.repeat(temp_pairs, 8, axis=0)

        temp_boards = np.vstack((temp_boards, -1*temp_boards))
        temp_masks = np.repeat(temp_masks, 2, axis=0)
        temp_values = np.vstack((temp_values, 1-temp_values))
        temp_pairs = np.vstack((temp_pairs, np.flip(temp_pairs, axis=1)))
        temp_outcome = np.vstack((temp_outcome, self.scale_factor-temp_outcome))
          
        self.model.fit([temp_boards, temp_pairs], temp_outcome, epochs=1, batch_size=batch_size, shuffle=True, validation_split=.1)

  def get_node_value(self, graph, node):
    if graph.nodes(data=True)[node]["visits"] != 0:
      return graph.nodes(data=True)[node]["outcomes"] / graph.nodes(data=True)[node]["visits"]
    else:
      return np.inf

  def getValues(self, boards, pairs, player_num, graph, train=True, random_agent=False):

    n = boards.shape[0]
    # print(board_states.shape)

    next_player = 0 if player_num == 1 else 1
    players_list = [player_num] * n
    curr_pairs = pairs[:, next_player].astype(int)
    last_pairs = pairs[:, player_num].astype(int)
    inp2 = np.array(list(zip(curr_pairs, last_pairs)))

    # # For inference, do not mask out anything, unlike when training where we mask out unexplored moves
    # # Masking out illegal moves will be done later
    pred_masks = np.ones((n, 19, 19, 1))

    if train:
      node_list = [multi.make_hashable((x[0], x[1], x[2])) for x in list(zip(boards.astype(int), last_pairs, curr_pairs))]

      l = [(edge[0], tuple(edge[2]["move"]), self.get_node_value(graph, edge[1]), graph.nodes(data=True)[edge[1]]["visits"]) for edge in list(graph.out_edges(node_list, data=True))]
      # print(len(l))
      l += [(node, None, 0) for node in node_list]

      for data in l:
        if data[0] not in self.node_values.keys():

          self.node_values[data[0]] = (np.zeros(shape=(19,19)), np.zeros(shape=(19,19)), np.zeros(shape=(19,19)))
          
          if data[1] is not None:
            # Add outcome score for child
            self.node_values[data[0]][0][data[1]] = data[2]
            # Add 1 to indicate that a node has been visited for masking training later
            self.node_values[data[0]][1][data[1]] = 1
            # Update visit count for particular child node
            self.node_values[data[0]][2][data[1]] = data[3]

        else:
          if data[1] is not None:
            # Add outcome score for child
            self.node_values[data[0]][0][data[1]] = data[2]
            # Add 1 to indicate that a node has been visited for masking training later
            self.node_values[data[0]][1][data[1]] = 1
            # Update visit count for particular child node
            self.node_values[data[0]][2][data[1]] = data[3]

      if not random_agent:

        parent_count_array = np.array([graph.nodes(data=True)[node]["visits"] if node in graph else 0 for node in node_list]).reshape((n, 1, 1, 1))
        child_count_array = np.array([self.node_values[node][2] for node in node_list]).reshape((n, 19, 19, 1))
        masks = np.array([self.node_values[node][1] for node in node_list])
        outcome_array = np.array([self.node_values[node][0] for node in node_list]).reshape((n, 19, 19, 1))

        policy_preds = self.model.predict([boards, inp2])

        ucb_score = np.array([outcome_array + self.c * (policy_preds / self.scale_factor) * np.sqrt(np.divide(np.log(parent_count_array + .0001), child_count_array + .000000001))]).reshape((n, 19, 19))

        np.nan_to_num(ucb_score, copy=False, nan=10000000, posinf=10000000, neginf=-10000000)
        # print("FINITE UCB_SCORE COUNT:", np.sum((ucb_score != 10000000)))
        
        return ucb_score
      
      else:

        return np.random.uniform(0,1,size=(n, 19, 19))
      
    else:

      if not random_agent:

        policy_preds = self.model.predict([boards, inp2])
        np.nan_to_num(policy_preds, copy=False, nan=10000000, posinf=10000000, neginf=-10000000)
        return policy_preds.reshape(n, 19, 19)

      else:

        return np.random.uniform(0,1,size=(n, 19, 19))




In [ ]:
# REPLACE WITH YOUR PATH FOR SAVING MODELS
path = "models"

In [ ]:
# max_num = max([int(d.split("_")[-1]) for d in os.listdir("models")])

# agent0 = Agent(agent_path=f"models/pente_ai_{max_num}")

agent0 = Agent()

multi = MultiPente(n_games = 1000)

num_batches = 10

path = "graph.pkl"

# Start with random playouts to train on
# for batch in range(1):
#   multi.play_train(agent0, agent0, make_random_moves=[True, True], num_games=5000)

# EDIT NUMBER OF BATCHES TO RUN HERE.
# EVEN ONE OR TWO WILL TAKE A FAIR BIT OF TIME.
num_batches = 500

agent_win_rate = []
batches = []

# Then add "intelligent" agent playouts
for batch in range(num_batches):

  print("BATCH", batch)
  multi.play_train(agent0, agent0, num_games=256)

  print("===============")
  print("GRAPH DETAILS")
  print(multi.graph)
  print("===============")
  print("===============")
  print("BATCH", batch)
  print("===============")

  # zerowinrate, onewinrate = multi.play_test(agent0, agent0, make_random_moves=[True, False], num_games=512)
  # agent_win_rate.append(onewinrate)
  # batches.append(batch)

  if batch % 5 == 0:
    keras.models.save_model(agent0.model, f"{path}/pente_ai_{batch}")

  # plt.scatter(x=batches, y=agent_win_rate)
  # plt.savefig(f"plots/{batch}.png")

  agent0.fit_on_graph(multi.graph, num_epochs=3, batch_size=512, sample_size=10000)

  

# with open(path, "wb") as f:
#   pickle.dump(multi.graph, f)

# keras.models.save_model(agent0.model, f"drive/MyDrive/pente_ai.h5")
# agent0.fit_on_graph(multi.graph, num_epochs=1000)
# keras.models.save_model(agent0.model, f"drive/MyDrive/pente_ai.h5")

# nx.draw(multi.graph, node_size=10)

# roots = {n for n,d in multi.graph.in_degree() if d==0}

# len(roots)


# plt.show()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 19, 19, 1)]  0           []                               
                                                                                                  
 gaussian_noise_10 (GaussianNoi  (None, 19, 19, 1)   0           ['input_3[0][0]']                
 se)                                                                                              
                                                                                                  
 conv2d_5 (Conv2D)              (None, 17, 17, 128)  1280        ['gaussian_noise_10[0][0]']      
                                                                                                  
 gaussian_noise_11 (GaussianNoi  (None, 17, 17, 128)  0          ['conv2d_5[0][0]']         

<ipython-input-7-56947904ffd0>:250: RuntimeWarning: invalid value encountered in sqrt
  ucb_score = np.array([outcome_array + self.c * (policy_preds / self.scale_factor) * np.sqrt(np.divide(np.log(parent_count_array + .0001), child_count_array + .000000001))]).reshape((n, 19, 19))


8/8 [==============================] - 0s 47ms/step
TAKING TURN 1
8/8 [==============================] - 0s 48ms/step
TAKING TURN 2
8/8 [==============================] - 1s 89ms/step
TAKING TURN 3
8/8 [==============================] - 1s 109ms/step
TAKING TURN 4
8/8 [==============================] - 1s 89ms/step
TAKING TURN 5
8/8 [==============================] - 1s 98ms/step
TAKING TURN 6
8/8 [==============================] - 1s 123ms/step
TAKING TURN 7
8/8 [==============================] - 1s 97ms/step
TAKING TURN 8
8/8 [==============================] - 1s 85ms/step
TAKING TURN 9
8/8 [==============================] - 1s 73ms/step
TAKING TURN 10
8/8 [==============================] - 1s 68ms/step
TAKING TURN 11
8/8 [==============================] - 1s 97ms/step
TAKING TURN 12
8/8 [==============================] - 0s 54ms/step
TAKING TURN 13
8/8 [==============================] - 1s 68ms/step
TAKING TURN 14
8/8 [==============================] - 0s 49ms/step
TAKING TURN 15
8/

(9935, 19, 19, 1)
280/280 [==============================] - 841s 3s/step - loss: 0.2473 - val_loss: 0.2944
(9940, 19, 19, 1)
280/280 [==============================] - 836s 3s/step - loss: 0.2466 - val_loss: 0.2934
(9945, 19, 19, 1)
 53/280 [====>.........................] - ETA: 10:46 - loss: 0.2460